In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip ml-100k.zip

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# AutoEncoder

## Pre-Processing

In [34]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t', names=['user_id', 'movie_id', 'rating', 'time'])
test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t', names=['user_id', 'movie_id', 'rating', 'time'])
training_set.head()

,user_id,movie_id,rating,time
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [35]:
matrix_train = training_set.groupby(['user_id', 'movie_id'])['rating'].mean().unstack().fillna(0)
matrix_test = test_set.groupby(['user_id', 'movie_id'])['rating'].mean().unstack().fillna(0)
matrix_train.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,0.0,4.0,1.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
nb_movies = len(matrix_train.columns)
nb_users = len(matrix_train)
training_set = np.array(matrix_train, dtype='int')
test_set = np.array(matrix_test, dtype='int')
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)
print(training_set.size())

torch.Size([943, 1650])


## Model Training

In [41]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)

In [49]:
nb_epoch = 5
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.

    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-9)
            loss.backward()
            train_loss += np.sqrt(loss.data * mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))

epoch: 1loss: tensor(1.7622)
epoch: 2loss: tensor(1.0947)
epoch: 3loss: tensor(1.0526)
epoch: 4loss: tensor(1.0379)
epoch: 5loss: tensor(1.0306)


## Evaluation

In [ ]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9513)


# Boltzman Machine (Bias within Sytem)

* Boltzmann Machine is a generative unsupervised model, which involves learning a probability distribution from an original dataset and using it to make inferences about never before seen data.

* Boltzmann Machine doesn’t expect input data, it generates data. Neurons generate information regardless they are hidden or visible.


https://medium.com/machine-learning-researcher/boltzmann-machine-c2ce76d94da5#:~:text=Boltzmann%20Machine%20consists%20of%20a,Machine%20is%20trying%20to%20minimize.

## Pre-processing

In [16]:
colnames = ['user_id', 'movie_id', 'rating', 'time']
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t', header=None, names=colnames)
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t', header=None, names=colnames)
full_set = pd.concat([training_set, test_set])
full_set['time'] = pd.to_datetime(full_set['time'], unit='s')
full_set.head()

,user_id,movie_id,rating,time
0,1,1,5,1997-09-22 22:02:38
1,1,2,3,1997-10-15 05:26:11
2,1,3,4,1997-11-03 07:42:40
3,1,4,3,1997-10-15 05:25:19
4,1,5,3,1998-03-13 01:15:12


In [17]:
nb_users = full_set['user_id'].nunique()
nb_movies = full_set['movie_id'].nunique()
training_set['rating'] = training_set['rating'].apply(lambda x: -1 if x < 1 else(0 if x < 3 else 1))
test_set['rating'] = test_set['rating'].apply(lambda x: -1 if x < 1 else(0 if x < 3 else 1))
training_set = training_set.groupby(['user_id', 'movie_id'])['rating'].max().unstack().fillna(-1)
test_set = test_set.groupby(['user_id', 'movie_id'])['rating'].max().unstack().fillna(-1)
test_set = test_set.reindex(columns = training_set.columns.tolist()).fillna(-1)

print(training_set.shape)
print(test_set.shape)
training_set.head()

(943, 1650)
(459, 1650)


movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,0.0,1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [18]:
training_set = np.array(training_set, dtype='int')
test_set = np.array(test_set, dtype='int')
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

print(training_set.size(), test_set.size())

torch.Size([943, 1650]) torch.Size([459, 1650])


## Model Training

In [19]:
# boltman machine class
class RBM():
    '''
    Parameters:
    nv: int. Number of Visible Nodes, supposed to be the size of data
    nh: int. Number of Hidden Nodes
    '''

    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)

    def sample_h(self, x):
        '''
        To get probability of hidden node given visible node
        Parameters:
        x: int. Vector of neurons
        '''    
        # [m, item_len] x [item_len, m] --> [m, m]
        wx = torch.mm(x, self.W.t())

        # [1, m] --> [m, m]
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)

        # Draws binary random numbers from Bernoulli distribution
        # [m, m], [m, m]
        return p_h_given_v, torch.bernoulli(p_h_given_v)

    def sample_v(self, y):
        # [m, m] x [m, item_len] --> [m, item_len]
        wy = torch.mm(y, self.W)

        # [m, item_len]
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)

    def train(self, v0, vk, ph0, phk):
        # contrast divergence algorithm
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

batch_size = 100
rbm = RBM(training_set.size()[1], 100)

### testing
batch_size = 100
vk = training_set[: batch_size]
v0 = training_set[: batch_size]
ph0, _ = rbm.sample_h(v0)
print("ph0 size: ", ph0.size())

for k in range(10):
    _, hk = rbm.sample_h(vk)
    _, vk = rbm.sample_v(hk)
    print("vk size: ", vk.size())
    vk[v0 < 0] = v0[v0 < 0]
    break

phk, _ = rbm.sample_h(vk)
print("phk size: ", phk.size())
rbm.train(v0, vk, ph0, phk)

train_loss = torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
print(train_loss)

ph0 size:  torch.Size([100, 100])
vk size:  torch.Size([100, 1650])
phk size:  torch.Size([100, 100])
tensor(0.4479)


In [21]:
# model training
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.0
    
    # Gibb Sampling
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user : id_user + batch_size]
        v0 = training_set[id_user : id_user + batch_size]
        ph0,_ = rbm.sample_h(v0)
        # monte carlo
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

## Evaluation

In [ ]:
# testing
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _, h = rbm.sample_h(v)
        _, v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))